In [2]:
from get_data import get_data
import pandas as pd
from random import sample
import re
import os

X, y = get_data()


X = pd.DataFrame(X, columns = ["reviewText"])
y = pd.Series(y)


y = y.where(X.reviewText.str.contains(r'[a-zA-Z]'))
X = X.where(X.reviewText.str.contains(r'[a-zA-Z]'))
X = X.reviewText.str.replace('\n', ' ')



ranges = [100, 500, 2000]

neg = X[y == 0]
pos = X[y == 1]

for r in ranges:
    print('\nProducing subset for size:', r)
    r = r*10
    text_neg = sample(list(neg), r//2)
    text_neg = ["0"+x for x in text_neg]
    text_pos = sample(list(pos), r//2)
    text_pos = ["1"+x for x in text_pos]
    text = text_neg+text_pos
    
    pattern1 = re.compile(r'([^0-9a-zA-Z\s])\1+(?=[a-z0-9A-Z])')
    pattern2 = re.compile(r'([^0-9a-zA-Z\s])\1+')
    pattern3 = re.compile(r'<[^>]>')
    for i, t in enumerate(text):
        t = re.sub(pattern1, r'\1 ', t)
        t = re.sub(pattern2, r'\1', t)
        t = re.sub(pattern3, r'', t)
        text[i] = t


    # fix "...blabl -> ... blabl" and reduce "... -> ."
    s_before = len(text)
    text[:] = filter(lambda sen: len(sen) <= 35, text)
    print('Found short sentences:',len(text))
    
    r = r//10
    df = get_data("n_{}".format(r))
    c = Counter(df["sentiment"])
    missing_neg = r//2 - c["0"]
    missing_pos = r//2 - c["1"]
    #"incase one is negative"
    if missing_neg < 0:
        missing_pos -= abs(missing_neg)
        missing_neg = 0
    elif missing_pos < 0:
        missing_neg -= abs(missing_pos)
        missing_pos = 0
    cwd = os.getcwd().replace("\\", "/") 
    with open(f'../Data/subsets/n_{r}.txt', 'a') as f:
        for i in text:
            label = i[0]
            hue = i[1:]
            if label == "0" and missing_neg > 0:
                f.write(f'{label}\t{hue}\n')
                missing_neg -= 1
            if label == "1" and missing_pos > 0:
                f.write(f'{label}\t{hue}\n')
                missing_pos -= 1

    


Producing subset for size: 100
Found short sentences: 201

Producing subset for size: 500
Found short sentences: 951

Producing subset for size: 2000
Found short sentences: 3730


In [3]:
from get_data import get_data
from collections import Counter
ranges = 500
df = get_data(type = "n_500")
c = Counter (df["sentiment"])
missing_neg = ranges//2 - c["0"]
missing_pos = ranges//2 - c["1"]

print(missing_neg, missing_pos)
    
        


0 0


In [18]:
X, Y = get_data()

In [21]:
import pandas as pd

In [28]:
X = pd.DataFrame(X, columns = ["reviewText"])

In [50]:
cwd = os.getcwd().replace("\\", "/")
f'{cwd}/Data/subsets/n_{r}_for_gpt.txt'

'C:/Users/Vikto/Desktop/meh/NLPPhard/modules/Data/subsets/n_100_for_gpt.txt'

In [9]:
df = get_data("n_100")

In [12]:
df

,sentiment,reviewText
0,0,It did not download easily and it still have n...
1,1,"['0Loosing their touch.', '0Wonderful cd. Ama..."
2,0,"no windows phone support, last thing im buying..."
3,0,"""Arkansas"" is the backside filler of 45 single..."
4,1,Love this song and the sound quality is great.
...,...,...
95,1,"['0Loosing their touch.', '0Wonderful cd. Ama..."
96,0,"Sounds like Sam Sparro's ""Black and Gold"" but ..."
97,0,It's not the version I heard when I was little...
98,1,Great variety of songs with this album. Nice t...
